In [60]:
from langchain_ollama.llms import OllamaLLM
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from langchain_core.messages import HumanMessage, SystemMessage, trim_messages, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import (BaseChatMessageHistory, InMemoryChatMessageHistory)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

In [2]:
load_dotenv()

True

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [35]:
model_openAi = ChatOpenAI(model="gpt-3.5-turbo")
model_ollama = OllamaLLM(model="gemma2")

In [5]:
model_ollama.invoke([HumanMessage("Hola como estas?")])

"As an AI, I don't have feelings or experiences like humans do. But thanks for asking!\n\nHow are you doing today? 😊"

In [6]:
model_ollama.invoke([HumanMessage("cual es mi nombre?")])

'Como modelo de lenguaje, no tengo acceso a información personal como tu nombre.\n\nPara que pueda ayudarte con tu pregunta, por favor, dime tu nombre. 😊'

In [7]:
model_ollama.invoke(
    [
        HumanMessage("Cual es el nombre de la persona que me atiende?"),
        SystemMessage("Mi nombre es Gemma2"),
        HumanMessage("Gemma2, tienes algun hermano?"),
    ]
)

'Como modelo lingüístico de IA, no tengo hermanos ni familia. Soy un programa informático diseñado para ayudar a las personas con tareas como generar texto, traducir idiomas y responder preguntas.  \n\n¿Hay algo más en lo que pueda ayudarte? 😊 \n'

# Message History

In [9]:
store = {}

In [10]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
    

In [18]:
with_message_history = RunnableWithMessageHistory(model_openAi, get_session_history)

In [19]:
## creando un session id
config_id = {"configurable": {"session_id": "1234"}}

In [20]:
response = with_message_history.invoke(
    [HumanMessage("que tamaño de monitor es recomentable para trabajar?")],
    config=config_id
    
)

In [21]:
response.content

'El tamaño recomendable de un monitor para trabajar depende de las necesidades y preferencias del usuario. Generalmente, se considera que un monitor de 24 pulgadas es adecuado para la mayoría de las tareas de trabajo, ya que proporciona suficiente espacio en pantalla para multitareas y una buena visualización de contenido.\n\nSin embargo, si se requiere de más espacio en pantalla, se puede optar por monitores de 27 pulgadas o incluso de 32 pulgadas, que ofrecen una mayor área de visualización y son ideales para tareas que requieren trabajar con múltiples ventanas abiertas al mismo tiempo.\n\nPor otro lado, si se necesita un monitor más portátil y con menor espacio ocupado en el escritorio, se pueden considerar monitores de 21 o 22 pulgadas.\n\nEn resumen, el tamaño recomendable de un monitor para trabajar dependerá de las necesidades específicas de cada usuario, pero en general, se considera que un monitor de 24 pulgadas es una buena opción para la mayoría de las personas.'

In [22]:
response = with_message_history.invoke(
    [HumanMessage("que configuracion es recomendable para un monitor de 32 pulgadas?")],
    config=config_id
    
)
response.content

'Para un monitor de 32 pulgadas, se recomienda una resolución de al menos 2560 x 1440 píxeles (QHD) para garantizar una buena calidad de imagen y una visualización nítida de los contenidos. Esto proporcionará una mayor densidad de píxeles en comparación con monitores de menor tamaño, lo que resultará en una experiencia visual más inmersiva.\n\nAdemás, es importante considerar la distancia a la que se va a utilizar el monitor. Para un monitor de 32 pulgadas, se recomienda colocarlo a una distancia de aproximadamente 80-100 centímetros del usuario para una visualización cómoda y ergonómica.\n\nEn cuanto a la configuración de color, se sugiere calibrar el monitor para obtener una reproducción precisa de los colores. Se puede utilizar un dispositivo de calibración de color o ajustar manualmente los ajustes de brillo, contraste y temperatura de color para lograr una imagen equilibrada y natural.\n\nEn resumen, para un monitor de 32 pulgadas, se recomienda una resolución de al menos 2560 x 1

# Promptemplates

In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Actua como un asistente de desarrollo de software y responde a las preguntas de los usuarios",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [25]:
chain = prompt | model_openAi

In [26]:
response = chain.invoke({"messages": [HumanMessage("como puedo instalar python?")]})
response.content

'Para instalar Python en tu computadora, sigue estos pasos:\n\n1. Ve al sitio web oficial de Python en https://www.python.org/downloads/\n2. Descarga la última versión estable de Python para tu sistema operativo (Windows, macOS o Linux).\n3. Ejecuta el instalador descargado y sigue las instrucciones en pantalla.\n4. Durante la instalación, asegúrate de marcar la casilla que dice "Add Python to PATH" (Agregar Python al PATH) para poder usar Python desde la línea de comandos.\n5. Una vez finalizada la instalación, puedes verificar que Python se ha instalado correctamente abriendo una terminal (en Windows, puedes abrir el símbolo del sistema o PowerShell) y escribiendo `python --version` o `python3 --version`.\n\n¡Listo! Ahora tienes Python instalado en tu computadora y puedes empezar a programar en este lenguaje.'

In [27]:
#agregando message history al prompt template
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [28]:
config_id_session = {"configurable": {"session_id": "789"}}

In [29]:
response = with_message_history.invoke(
    [HumanMessage("usando conda puedo instalar diferentes versiones de python?")],
    config=config_id_session
)

In [30]:
response.content

'Sí, con Conda puedes crear entornos virtuales con diferentes versiones de Python. Por ejemplo, puedes crear un entorno con Python 3.6 y otro con Python 3.8. De esta manera, puedes trabajar en proyectos que requieran versiones específicas de Python sin que entren en conflicto entre sí. Aquí te dejo un ejemplo de cómo crear un entorno con una versión específica de Python utilizando Conda:\n\n1. Para crear un nuevo entorno con una versión específica de Python, puedes utilizar el siguiente comando en la terminal:\n\n```\nconda create -n mi_entorno python=3.6\n```\n\nEste comando creará un nuevo entorno llamado "mi_entorno" con Python 3.6 instalado.\n\n2. Para activar el entorno recién creado, puedes utilizar el siguiente comando:\n\n```\nconda activate mi_entorno\n```\n\nUna vez activado el entorno, puedes instalar las librerías y paquetes necesarios para tu proyecto en ese entorno específico.\n\nRecuerda que es importante mantener los entornos virtuales separados para evitar conflictos e

In [31]:
response = with_message_history.invoke(
    [HumanMessage("y puedo usala con visual studio code?")],
    config=config_id_session
)

In [32]:
response.content

'Sí, puedes utilizar Conda con Visual Studio Code para gestionar entornos virtuales y trabajar en proyectos de Python de manera más organizada. Aquí te explico cómo puedes configurar Visual Studio Code para que reconozca y utilice entornos virtuales de Conda:\n\n1. Abre Visual Studio Code y asegúrate de tener instalada la extensión "Python" de Microsoft.\n\n2. En la parte inferior izquierda de la ventana de Visual Studio Code, verás un botón que indica el intérprete de Python actualmente seleccionado. Haz clic en este botón y selecciona "Seleccionar intérprete".\n\n3. En la lista de intérpretes que aparece, busca la opción "Enter interpreter path..." y haz clic en ella.\n\n4. En la ventana que se abre, introduce la ruta al intérprete de Python que se encuentra dentro del entorno virtual que creaste con Conda. Por lo general, la ruta tendrá una estructura similar a esta: `C:\\Users\\tu_usuario\\Anaconda3\\envs\\mi_entorno\\python.exe`.\n\n5. Una vez seleccionada la ruta correcta, Visual

### Promptemplate idiomas

In [33]:
prompt_language = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Actua como un asistente de lenguaje natural y responde a las preguntas de los usuarios in {language}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [49]:
parser_str = StrOutputParser()

In [50]:
chain = prompt_language | model_ollama | parser_str

In [43]:
response = chain.invoke({"messages": [HumanMessage("como decir: hola como estas")], "language": "portugues"})
response

'Você pode dizer "Olá, como você está?" \n\nEssa é uma forma comum e educada de cumprimentar alguém em português. 😊\n\n\nLet me know if you have any other questions! 😄  \n'

In [44]:
# con historial de conversacion
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

In [47]:
config_id_session = {"configurable": {"session_id": "456"}}

In [51]:
response = with_message_history.invoke(
    {"messages": [HumanMessage("hola como estas, soy miguel y tu?")], "language": "portugues"},
    config=config_id_session
)
response

'Olá, como você está? Eu sou o Miguel. E você? \n\n\nI hope this is helpful! Let me know if you need anything else. 😊  \n'

In [52]:
config_id_session = {"configurable": {"session_id": "45622"}}

In [53]:
response = with_message_history.invoke(
    {"messages": [HumanMessage("hola como estas, soy miguel y tu?")], "language": "portugues"},
    config=config_id_session
)
response

'Olá Miguel, tudo bem! Eu estou ótimo, obrigado por perguntar.  \n\nE você, como está? 😊 \n'

# Manejo del historial de conversacion

In [73]:
trimmer = trim_messages(
    max_tokens=400,
    strategy="last",
    token_counter=model_ollama,
    include_system=True,
    allow_partial=True,
    start_on="human"  
)

In [80]:

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content="hi! I'm bob"),
 AIMessage(content='hi!'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [87]:
pregunta1 = HumanMessage(content="cual problema de matemaricas te pregunte?")
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt_language
    | model_openAi
    | parser_str 
)
response = chain.invoke(
    {
        "messages": messages + [pregunta1],
        "language": "espanol"
    }
)
messages.append(pregunta1)
messages.append(response)
response

'Me preguntaste cuánto es 2 + 2. ¡Espero haber respondido correctamente!'

In [78]:
pregunta2= HumanMessage(content="cual es mi nombre")
response = chain.invoke(
    {
        "messages": messages + [pregunta2],
        "language": "espanol"
    }
)
messages.append(pregunta2)
messages.append(response)
response

'¡Sí, me encantaría! ¿Qué otro problema quieres que intente resolver? 😊  \n'

In [69]:
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='¡Tienes razón! Te pregunté cuál era el resultado de 2 + 2. 😊  \n\n¿Hay alguna otra pregunta que pueda ayudarte a resolver? \n')]

In [90]:
pregunta3 = HumanMessage(content="cuantame un chiste, sobre algo que me haya pasado a mi")
for r in chain.stream(
     {
        "messages": messages + [pregunta3],
        "language": "espanol"
    }
):
    print(r, end="|")

|¡|Cl|aro|!| ¿|Por| qué| el| hel|ado| de| vain|illa| se| p|uso| tr|iste|?| ¡|Por|que| Bob| se| lo| com|ió| todo|!| 🍦|🤣||